In [41]:
library(limma)

In [42]:
# Charger le pseudo-bulk
pb <- read.csv("pseudobulk_matrix.csv")

# Séparer l’expression et les métadonnées
expr <- pb[, !(colnames(pb) %in% c("class", "donor_id", "disease"))]
expr <- as.matrix(expr)

meta <- data.frame(
  class = pb$class,
  donor = pb$donor_id,
  disease = factor(pb$disease)
)
rownames(expr) <- paste(pb$class, pb$donor_id, pb$disease, sep="_")


In [43]:
# Liste des cell-types
celltypes <- unique(meta$class)

# Création d’un dossier de sortie
dir.create("DE_results", showWarnings = FALSE)

meta$class <- gsub("[/ ]+", "_", meta$class)

for (ct in celltypes) {
  
  cat("\n=== Cell type :", ct, "===\n")
  
  # Sélection des donors de ce cell type
  idx <- meta$class == ct
  expr_ct <- expr[idx, ]
  meta_ct <- meta[idx, ]
  
  expr_ct <- expr_ct[, colMeans(expr_ct, na.rm=TRUE) > 0.01 & apply(expr_ct, 2, var) > 1e-5]


  print(table(meta_ct$disease))

  # Relevel pour que CTRL = reference
  meta_ct$disease <- factor(meta_ct$disease, levels = c("normal", "dementia || Alzheimer disease",
                                                       "dementia || Parkinson disease"))
  
  # Design matrix
  design <- model.matrix(~ 0 + disease, data=meta_ct)
  colnames(design) <- c("CTRL", "AD", "PD")
  
  cat("Design matrix OK\n")
  
  # modèle limma-trend
  fit <- lmFit(t(expr_ct), design)

  contrasts <- makeContrasts(
    AD_vs_CTRL = AD - CTRL,
    PD_vs_CTRL = PD - CTRL,
    AD_vs_PD   = AD - PD,
    levels = design
  )
  
  fit2 <- contrasts.fit(fit, contrasts)
  fit2 <- eBayes(fit2, trend = TRUE)
  
  # cell type nettoyé
  ct_clean <- gsub("[/ ]+", "_", ct)

# Export
  write.csv(topTable(fit2, coef="AD_vs_CTRL", number=Inf),
            paste0("DE_results/DE_", ct_clean, "_ADvsCTRL.csv"))
  
  write.csv(topTable(fit2, coef="PD_vs_CTRL", number=Inf),
            paste0("DE_results/DE_", ct_clean, "_PDvsCTRL.csv"))
  
  write.csv(topTable(fit2, coef="AD_vs_PD", number=Inf),
            paste0("DE_results/DE_", ct_clean, "_ADvsPD.csv"))
}


=== Cell type : Astro ===

dementia || Alzheimer disease dementia || Parkinson disease 
                           17                            17 
                       normal 
                           17 
Design matrix OK

=== Cell type : EN ===

dementia || Alzheimer disease dementia || Parkinson disease 
                           17                            17 
                       normal 
                           17 
Design matrix OK

=== Cell type : Endo ===

dementia || Alzheimer disease dementia || Parkinson disease 
                           17                            17 
                       normal 
                           17 
Design matrix OK

=== Cell type : IN ===

dementia || Alzheimer disease dementia || Parkinson disease 
                           17                            17 
                       normal 
                           17 
Design matrix OK

=== Cell type : Immune ===

dementia || Alzheimer disease dementia || Parkinson disease 
 

In [47]:
dim(expr_ct)

[1]    51 33828

- aucun donneur n’a plusieurs diseases
- pas de duplication artificielle
- cellules présentes seulement pour certaines conditions
- distribution logique (tous les donneurs n’ont pas toutes les cell types)

Certains gènes ont une variance extrêmement faible entre donors dans un cell type donné.

C’est normal en pseudo-bulk log-normalisé
- encore plus normal quand :
- un cell type a peu de donors (ex : 2–3 en PD)
- les valeurs sont des moyennes log-normalisées
- beaucoup de gènes sont presque non exprimés

In [48]:
head(meta)
head(expr)
table(meta$class, meta$disease)
apply(expr, 1, sd)[1:20]  # 20 gènes au hasard

,class,donor,disease
,<chr>,<chr>,<fct>
1,Astro,Donor_31,dementia || Alzheimer disease
2,Astro,Donor_31,dementia || Parkinson disease
3,Astro,Donor_31,normal
4,Astro,Donor_228,dementia || Alzheimer disease
5,Astro,Donor_228,dementia || Parkinson disease
6,Astro,Donor_228,normal


,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,⋯,ENSG00000288600,ENSG00000288602,ENSG00000288605,ENSG00000288606,ENSG00000288611,ENSG00000288612,ENSG00000288622,ENSG00000288637,ENSG00000288642,ENSG00000288643
Astro_Donor_31_dementia || Alzheimer disease,71,7,242,173,139,1,4,64,370,84,⋯,0,98,37,6,0,21,4,3,15,13
Astro_Donor_31_dementia || Parkinson disease,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Astro_Donor_31_normal,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Astro_Donor_228_dementia || Alzheimer disease,63,1,258,197,123,2,1,55,630,100,⋯,1,119,25,10,1,30,3,5,10,17
Astro_Donor_228_dementia || Parkinson disease,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Astro_Donor_228_normal,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


        
         dementia || Alzheimer disease dementia || Parkinson disease normal
  Astro                             17                            17     17
  EN                                17                            17     17
  Endo                              17                            17     17
  Immune                            17                            17     17
  IN                                17                            17     17
  Mural                             17                            17     17
  Oligo                             17                            17     17
  OPC                               17                            17     17

Astro_Donor_31_dementia || Alzheimer disease 
                                     1796.998 
 Astro_Donor_31_dementia || Parkinson disease 
                                        0.000 
                        Astro_Donor_31_normal 
                                        0.000 
Astro_Donor_228_dementia || Alzheimer disease 
                                     1797.759 
Astro_Donor_228_dementia || Parkinson disease 
                                        0.000 
                       Astro_Donor_228_normal 
                                        0.000 
Astro_Donor_333_dementia || Alzheimer disease 
                                     2002.402 
Astro_Donor_333_dementia || Parkinson disease 
                                        0.000 
                       Astro_Donor_333_normal 
                                        0.000 
Astro_Donor_545_dementia || Alzheimer disease 
                                        0.000 
Astro_Donor_545_dementia || Parkinson disease 
                                     1211.420 
                       Astro_Donor_545_normal 
                                        0.000 
Astro_Donor_609_dementia || Alzheimer disease 
                                        0.000 
Astro_Donor_609_dementia || Parkinson disease 
                                        0.000 
                       Astro_Donor_609_normal 
                                     2309.980 
Astro_Donor_614_dementia || Alzheimer disease 
                                     1064.214 
Astro_Donor_614_dementia || Parkinson disease 
                                        0.000 
                       Astro_Donor_614_normal 
                                        0.000 
Astro_Donor_634_dementia || Alzheimer disease 
                                        0.000 
Astro_Donor_634_dementia || Parkinson disease 
                                        0.000

In [46]:
expr_ct.var

ERROR: Error: object 'expr_ct.var' not found
